In [12]:
import pandas as pd
import numpy as np
import pickle
import plotly.express as px
from datetime import datetime
from sklearn import preprocessing

In [13]:
sales = pd.read_excel('sales-pipeline.xlsx')
sales.head()

,Account,Opportunity_ID,Sales_Agent,SalesAgentEmailID,ContactEmailID,Deal_Stage,Product,Close_Value,Created Date,Close Date,DateDiff
0,Konex,3KR50X38,Corliss Cosme,corliss@piedpiper.com,delila@konex.com,Won,GTX Plus Basic,1255,2016-01-04,2016-05-24,141
1,Finjob,YFS5KLJJ,Rosalina Dieter,rosalina@piedpiper.com,belinda@finjob.com,Won,MG Special,45,2016-01-04,2016-11-02,303
2,Kinnamplus,N0ONCYVZ,Donn Cantrell,donn@piedpiper.com,monte@kinnamplus.com,Lost,MG Special,1054,2016-01-06,2016-12-07,336
3,Genco Pura Olive Oil Company,IBEUTCSW,James Ascencio,james@piedpiper.com,karole@gencopuraoliveoilcompany.com,In Progress,MG Advanced,1054,2016-01-06,2016-09-11,249
4,Initech,G8AO5FA2,Wilburn Farren,wilburn@piedpiper.com,alejandro@initech.com,Lost,MG Advanced,517,2016-01-06,2016-09-18,256


In [14]:
sales[(sales['SalesAgentEmailID'] == 'corliss@piedpiper.com') & (sales['ContactEmailID'] == 'delila@konex.com')]

,Account,Opportunity_ID,Sales_Agent,SalesAgentEmailID,ContactEmailID,Deal_Stage,Product,Close_Value,Created Date,Close Date,DateDiff
0,Konex,3KR50X38,Corliss Cosme,corliss@piedpiper.com,delila@konex.com,Won,GTX Plus Basic,1255,2016-01-04,2016-05-24,141
171,Konex,41MURAI5,Corliss Cosme,corliss@piedpiper.com,delila@konex.com,Won,GTX Basic,590,2016-07-01,2016-07-18,17
506,Konex,BP3I9G0F,Corliss Cosme,corliss@piedpiper.com,delila@konex.com,Won,GTX Plus Basic,1105,2017-01-07,2017-05-19,132
2149,Konex,AVZ1BG3K,Corliss Cosme,corliss@piedpiper.com,delila@konex.com,Lost,MG Advanced,3284,2017-03-22,2017-11-10,233
3328,Konex,3RJPZ634,Corliss Cosme,corliss@piedpiper.com,delila@konex.com,Won,GTXPro,4492,2017-05-18,2018-01-13,240
3807,Konex,2PP30ME4,Corliss Cosme,corliss@piedpiper.com,delila@konex.com,Lost,GTX Plus Basic,1180,2017-06-08,2019-01-06,577
4171,Konex,1KVGSF08,Corliss Cosme,corliss@piedpiper.com,delila@konex.com,Won,GTX Plus Pro,4989,2017-06-25,2019-08-21,787
6283,Konex,BG2656VA,Corliss Cosme,corliss@piedpiper.com,delila@konex.com,Won,GTX Plus Pro,5684,2017-09-30,2017-11-15,46
6476,Konex,M765WQ4W,Corliss Cosme,corliss@piedpiper.com,delila@konex.com,Won,GTXPro,5067,2017-10-09,2018-08-03,298
6638,Konex,HELQUZNO,Corliss Cosme,corliss@piedpiper.com,delila@konex.com,Won,GTX Basic,616,2017-10-16,2018-05-06,202


In [15]:
interx = pd.read_excel('interactions.xlsx')
interx.head()

,interactionID,fromEmailId,toEmailId,InteractionDate,InteractionType(Call/Email/SMS),Extracted Interaction Text
0,8842411,gladys@piedpiper.com,kristle@dontechi.com,2016-01-04,Call,I thoroughly enjoyed your presentation yesterd...
1,8015509,donn@piedpiper.com,orpha@umbrellacorporation.com,2016-01-04,Call,speaking with you and your staff will be a tre...
2,5976842,kary@piedpiper.com,claribel@goodsilron.com,2016-01-04,Email,"However, I am unable to join because of my fam..."
3,1471794,boris@piedpiper.com,gordon@scotfind.com,2016-01-04,Call,We are extremely sorry to inform you that we h...
4,4330320,darcel@piedpiper.com,benny@treequote.com,2016-01-04,Call,I don't think it's possible for me to meet you


In [16]:
row = sales.iloc[0]
from_, to, start, end = row['SalesAgentEmailID'], row['ContactEmailID'], row['Created Date'], row['Close Date']
list(interx[(interx['fromEmailId']==from_) & (interx['toEmailId']==to) & \
                   (interx['InteractionDate'] <= end) & (interx['InteractionDate'] >= start)]['Extracted Interaction Text'])

['We are writing from GHF regarding the previous discussions that we had about the business proposal that your company sent us. We are happy to let you know that GHF accepted your business proposal. Thank you for offering us the opportunity of working with you. We are looking forward to work with your company.',
 "I had asked you to send the progress on the project by the end of last week but I still haven't got it. Do send us the report asap. We will not be able to move forward with the deal unless we get it verified by our CEO. ",
 'I wish to commend your receptionist, Jany Dan, for the way she handled our problem yesterday. She was courteous and friendly as she made half a dozen telephone calls on my behalf, not giving up until she had traced the missing shipment. Ms. Dan is an asset to your organization and is part of the reason I look forward to doing business with you again in the future.',
 'I need to cancel the appointment I made to you on January 29, at 9:00 a.m.',
 'Congratul

In [23]:
interx.insert(6, 'Raw Texts', '')
for i in range(interx.shape[0]):
    text = interx['Extracted Interaction Text'].iloc[i].split()
    text = ''.join(text)
    interx.iat[i, -1] = text

In [24]:
interx.head()

,interactionID,fromEmailId,toEmailId,InteractionDate,InteractionType(Call/Email/SMS),Extracted Interaction Text,Raw Texts
0,8842411,gladys@piedpiper.com,kristle@dontechi.com,2016-01-04,Call,I thoroughly enjoyed your presentation yesterd...,Ithoroughlyenjoyedyourpresentationyesterdayeve...
1,8015509,donn@piedpiper.com,orpha@umbrellacorporation.com,2016-01-04,Call,speaking with you and your staff will be a tre...,speakingwithyouandyourstaffwillbeatremendousop...
2,5976842,kary@piedpiper.com,claribel@goodsilron.com,2016-01-04,Email,"However, I am unable to join because of my fam...","However,Iamunabletojoinbecauseofmyfamilyobliga..."
3,1471794,boris@piedpiper.com,gordon@scotfind.com,2016-01-04,Call,We are extremely sorry to inform you that we h...,Weareextremelysorrytoinformyouthatwehavedecide...
4,4330320,darcel@piedpiper.com,benny@treequote.com,2016-01-04,Call,I don't think it's possible for me to meet you,Idon'tthinkit'spossibleformetomeetyou


In [25]:
len(interx['Raw Texts'].unique())

181

In [26]:

le = preprocessing.LabelEncoder()
le.fit(interx['Raw Texts'])

le_texts = le.transform(interx['Raw Texts'])

interx['le_texts'] = le_texts

#le.classes_

#le.inverse_transform([0, 1, 2])

In [27]:
output = open('texts_encoder.pkl', 'wb')
pickle.dump(le, output)
output.close()

In [28]:
interx.head()

,interactionID,fromEmailId,toEmailId,InteractionDate,InteractionType(Call/Email/SMS),Extracted Interaction Text,Raw Texts,le_texts
0,8842411,gladys@piedpiper.com,kristle@dontechi.com,2016-01-04,Call,I thoroughly enjoyed your presentation yesterd...,Ithoroughlyenjoyedyourpresentationyesterdayeve...,74
1,8015509,donn@piedpiper.com,orpha@umbrellacorporation.com,2016-01-04,Call,speaking with you and your staff will be a tre...,speakingwithyouandyourstaffwillbeatremendousop...,179
2,5976842,kary@piedpiper.com,claribel@goodsilron.com,2016-01-04,Email,"However, I am unable to join because of my fam...","However,Iamunabletojoinbecauseofmyfamilyobliga...",24
3,1471794,boris@piedpiper.com,gordon@scotfind.com,2016-01-04,Call,We are extremely sorry to inform you that we h...,Weareextremelysorrytoinformyouthatwehavedecide...,134
4,4330320,darcel@piedpiper.com,benny@treequote.com,2016-01-04,Call,I don't think it's possible for me to meet you,Idon'tthinkit'spossibleformetomeetyou,55


In [29]:
def get_interx(from_, to, start, end):
    codes = list(interx[(interx['fromEmailId']==from_) & (interx['toEmailId']==to) & \
                   (interx['InteractionDate'] <= end) & (interx['InteractionDate'] >= start)]['le_texts'])
    return str(codes)[1:-1]

In [30]:
row = sales.iloc[0]
from_, to, start, end = row['SalesAgentEmailID'], row['ContactEmailID'], row['Created Date'], row['Close Date']
sample = get_interx(from_, to, start, end)
sample

'146, 59, 87, 66, 15, 72, 115, 112, 39, 63, 36, 107, 4, 34, 78, 72, 142, 40, 145, 76, 120, 17, 34, 108, 68, 20, 158, 102'

In [34]:
final = sales.copy()
final.columns

Index(['Account', 'Opportunity_ID', 'Sales_Agent', 'SalesAgentEmailID',
       'ContactEmailID', 'Deal_Stage', 'Product', 'Close_Value',
       'Created Date', 'Close Date', 'DateDiff'],
      dtype='object')

In [35]:
final = final[['Account', 'SalesAgentEmailID', 'ContactEmailID', 'Deal_Stage', 'Product', 'Close_Value', 'Created Date', 'Close Date', 'DateDiff']]
final.head()

,Account,SalesAgentEmailID,ContactEmailID,Deal_Stage,Product,Close_Value,Created Date,Close Date,DateDiff
0,Konex,corliss@piedpiper.com,delila@konex.com,Won,GTX Plus Basic,1255,2016-01-04,2016-05-24,141
1,Finjob,rosalina@piedpiper.com,belinda@finjob.com,Won,MG Special,45,2016-01-04,2016-11-02,303
2,Kinnamplus,donn@piedpiper.com,monte@kinnamplus.com,Lost,MG Special,1054,2016-01-06,2016-12-07,336
3,Genco Pura Olive Oil Company,james@piedpiper.com,karole@gencopuraoliveoilcompany.com,In Progress,MG Advanced,1054,2016-01-06,2016-09-11,249
4,Initech,wilburn@piedpiper.com,alejandro@initech.com,Lost,MG Advanced,517,2016-01-06,2016-09-18,256


In [36]:
final.insert(9, 'le_texts', '')
for i in range(final.shape[0]):
    row = sales.iloc[i]
    from_, to, start, end = row['SalesAgentEmailID'], row['ContactEmailID'], row['Created Date'], row['Close Date']
    final.iat[i, 9] = get_interx(from_, to, start, end)

In [37]:
final.head()

,Account,SalesAgentEmailID,ContactEmailID,Deal_Stage,Product,Close_Value,Created Date,Close Date,DateDiff,le_texts
0,Konex,corliss@piedpiper.com,delila@konex.com,Won,GTX Plus Basic,1255,2016-01-04,2016-05-24,141,"146, 59, 87, 66, 15, 72, 115, 112, 39, 63, 36,..."
1,Finjob,rosalina@piedpiper.com,belinda@finjob.com,Won,MG Special,45,2016-01-04,2016-11-02,303,"85, 153, 12, 62, 165, 113, 33"
2,Kinnamplus,donn@piedpiper.com,monte@kinnamplus.com,Lost,MG Special,1054,2016-01-06,2016-12-07,336,"35, 2, 148, 27, 43, 0, 96, 100, 132, 116, 138,..."
3,Genco Pura Olive Oil Company,james@piedpiper.com,karole@gencopuraoliveoilcompany.com,In Progress,MG Advanced,1054,2016-01-06,2016-09-11,249,"52, 37, 150, 118, 99, 133, 42"
4,Initech,wilburn@piedpiper.com,alejandro@initech.com,Lost,MG Advanced,517,2016-01-06,2016-09-18,256,"102, 119, 171, 157, 148"


In [38]:
final.to_excel('Label_encoded.xlsx')